# Meta-Modeling: Combine.jl

Idea adapted from Orchestra.jl work of Samuel Jenkins (https://github.com/svs14/Orchestra.jl)

In [52]:
using Combine.Util
using Combine.Transformers
import RDatasets

In [53]:
iris = RDatasets.dataset("datasets", "iris")
X = convert(Array, iris[[:SepalLength, :SepalWidth, :PetalLength, :PetalWidth]])
y = convert(Array, iris[:Species]);

# Split into training and test sets
(train_ind, test_ind) = holdout(size(X, 1), 0.3)

([59,65,107,111,133,30,106,58,29,6  …  60,123,56,105,130,113,43,147,75,37],[63,27,145,103,41,32,1,68,102,87  …  10,69,112,74,22,40,57,38,148,96])

In [54]:
prunedTreeLearner = PrunedTree()

Combine.Transformers.DecisionTreeWrapper.PrunedTree(nothing,Dict{Symbol,Any}(Pair{Symbol,Any}(:impl_options,Dict(:purity_threshold=>1.0)),Pair{Symbol,Any}(:output,:class)))

In [55]:
pipeline = Pipeline(Dict(
        :transformers => [
            OneHotEncoder(), # Encodes nominal features into numeric
            Imputer(), # Imputes NA values
            #StandardScaler(), # Standardizes features 
            prunedTreeLearner # Predicts labels on instances
        ]
    ))

Combine.Transformers.CombineTransformers.Pipeline(nothing,Dict{Symbol,Any}(Pair{Symbol,Any}(:transformers,Combine.Types.Transformer[Combine.Transformers.CombineTransformers.OneHotEncoder(nothing,Dict(:nominal_columns=>nothing,:nominal_column_values_map=>nothing)),Combine.Transformers.CombineTransformers.Imputer(nothing,Dict(:strategy=>mean)),Combine.Transformers.DecisionTreeWrapper.PrunedTree(nothing,Dict{Symbol,Any}(Pair{Symbol,Any}(:impl_options,Dict(:purity_threshold=>1.0)),Pair{Symbol,Any}(:output,:class)))]),Pair{Symbol,Any}(:transformer_options,nothing)))

In [56]:
# Train
fit!(pipeline, X[train_ind, :], y[train_ind]);

In [57]:
# Predict
predictions = transform!(pipeline, X[test_ind, :]);

In [58]:
sum(predictions .== y[test_ind])/length(predictions)*100

95.55555555555556

In [15]:
result = score(:accuracy, y[test_ind], predictions)
println(result)

86.66666666666667


In [16]:
function processModel(learner)
    iris = RDatasets.dataset("datasets", "iris")
    X = convert(Array, iris[[:SepalLength, :SepalWidth, :PetalLength, :PetalWidth]])
    y = convert(Array, iris[:Species]);
    (train_ind, test_ind) = holdout(size(X, 1), 0.3)
    pipeline = Pipeline(Dict(
            :transformers => [
                OneHotEncoder(), # Encodes nominal features into numeric
                Imputer(), # Imputes NA values
                #StandardScaler(), # Standardizes features 
                learner # Predicts labels on instances
            ]
        ))
    # Train
    fit!(pipeline, X[train_ind, :], y[train_ind]);
    # Predict
    predictions = transform!(pipeline, X[test_ind, :]);
    result = score(:accuracy, y[test_ind], predictions)
    return(result)
end

processModel (generic function with 1 method)

In [21]:
processModel(prunedTreeLearner)

91.11111111111111

In [24]:
adaLearner = DecisionStumpAdaboost(Dict(
  # Output to train against
  # (:class).
  :output => :class,
  # Options specific to this implementation.
  :impl_options => Dict(
    # Number of boosting iterations.
    :num_iterations => 7
  )
))
processModel(adaLearner)

91.11111111111111

In [26]:
rfLearner = RandomForest(Dict(
  :output => :class,
  :impl_options => Dict(
    :num_subfeatures => nothing,
    :num_trees => 10,
    :partial_sampling => 0.7
  )
))
processModel(rfLearner)

93.33333333333333

In [32]:
using ScikitLearn
@sk_import neighbors: KNeighborsClassifier
@sk_import svm: SVC

skLearner = SKLLearner(Dict(
  :output => :class,
  #:learner => "KNeighborsClassifier",
  :learner => "SVC",
  :impl_options => Dict()
))
processModel(skLearner)

95.55555555555556

In [45]:
voteLearner = VoteEnsemble(Dict(
  :output => :class,
  # Learners in voting committee.
  :learners => [RandomForest(),PrunedTree(), adaLearner,skLearner]
))
processModel(voteLearner)

95.55555555555556

In [46]:
bestLearner = BestLearner(Dict(
  :output => :class,
  :partition_generator => (X, y) -> kfold(size(X, 1), 5),
  :selection_function => (learner_partition_scores) -> findmax(mean(learner_partition_scores, 2))[2],      
  :score_type => Real,
  :learners => [PrunedTree(), DecisionStumpAdaboost(), RandomForest(),skLearner],
  :learner_options_grid => nothing
))
processModel(bestLearner)

97.77777777777777

In [48]:
stackLearner = StackEnsemble(Dict(
  :output => :class,
  :learners => [PrunedTree(), DecisionStumpAdaboost(), RandomForest(),skLearner,voteLearner,bestLearner],
  :stacker => RandomForest(),
  # Proportion of training set left to train stacker itself.
  :stacker_training_proportion => 0.3,
  :keep_original_features => false
))
processModel(stackLearner)

93.33333333333333

In [49]:
results=@parallel (vcat) for i=1:30
   processModel(stackLearner)
end
println("acc = ",round(mean(results))," +/- ",round(std(results)))

acc = 95.0 +/- 3.0


In [50]:
results

30-element Array{Float64,1}:
  97.7778
  97.7778
  97.7778
  97.7778
  95.5556
  95.5556
  91.1111
  95.5556
  95.5556
  93.3333
  91.1111
  93.3333
  91.1111
   ⋮     
  91.1111
  93.3333
  95.5556
  95.5556
  93.3333
 100.0   
  93.3333
  91.1111
  95.5556
  93.3333
  95.5556
  95.5556

In [18]:
# svmcrt = CRTLearner(Dict(
#   Output to train against
#   (:class).
#   :output => :class,
#   :learner => "rf",
#   :learner => "svmLinear2",
#   :learner => "rpart",
#   :learner => "lda",
#   :impl_options => Dict()
# ))

In [67]:
?kfold

search: kfold



No documentation found.

`Combine.Util.kfold` is a `Function`.

```
# 1 method for generic function "kfold":
kfold(num_instances, num_partitions) at /Users/ppalmes/.julia/v0.5/Combine/src/util.jl:36
```
